In [42]:
# this notebook is 1_Networks_Coding_TA_rev2_model-lg

#import os
import time

import pdfplumber
#import PyPDF2

import import_ipynb
from _functions_rev1 import *

import spacy
from spacy import displacy
from spacy.matcher import Matcher
import spacy_curated_transformers

from collections import Counter
import re

import pickle

In [29]:
raw_text_cleaned = ''

start_time = time.monotonic()

with pdfplumber.open("S3 Tool Changer.pdf") as pdf:
    for page in pdf.pages:
        # extract text
        text = page.extract_text()
        # cleaning text
        text = clean_text_3(text)
        raw_text_cleaned += text

print("--- %s minutes ---" % ((time.monotonic() - start_time)/60))
#raw_text_cleaned

--- 0.5270833333333333 minutes ---


In [30]:
nlp = spacy.load("en_core_web_lg")

In [31]:
start_time = time.monotonic()

#doc_ref = nlp(raw_text)
doc = nlp(raw_text_cleaned)

print("--- %s seconds ---" % (time.monotonic() - start_time))

--- 3.1879999998491257 seconds ---


In [32]:
sentence_spans = []
for sent in doc.sents:
    sentence_spans.append(sent)
#sentence_spans

In [33]:
phrase = r'(?=.*\b(tooling|tool)\b)(?=.*\b(adaptor|adaptors)\b)(?!.*\b(assembly|assemblies|assembly¼s)\b)'

In [34]:
list_matched_sentss = []
counter=0
for item in sentence_spans:
    if re.search(phrase, item.text, re.IGNORECASE) or re.search(phrase, item.text, re.IGNORECASE) or re.search(phrase, item.text, re.IGNORECASE):
        #print(counter,"\n",item)
        counter+=1
        list_matched_sentss.append(item)

In [35]:
list_matched_sents = list_matched_sentss
#filtration for Tooling Adaptor sents
list_matched_sents[2] = list_matched_sents[2][4:]
list_matched_sents[7] = list_matched_sents[7][2:]
list_matched_sents[8] = list_matched_sents[8][4:]
list_matched_sents = list_matched_sents[:-2]
list_matched_sents = list_matched_sents[2:]

#list_matched_sents

In [12]:
#displacy.render(list_matched_sents[0], style="dep", jupyter=True, options={'distance':80})

In [36]:
def decisionMaker(sent):
    wordsInSents = {}
    
    for token in sent:
        if spacy.explain(token.pos_) in ('noun', 'pronoun', 'proper noun'):
            wordsInSents[token.text] = 'object'
        if spacy.explain(token.pos_) in ('particle', 'auxiliary', 'verb'):
            wordsInSents[token.text] = 'action'
        if spacy.explain(token.pos_) in ('adposition', 'subordinating conjunction', 'coordinating conjunction'):
            wordsInSents[token.text] = 'positioning'
        if spacy.explain(token.pos_) in ('determiner', 'adverb', 'adjective', 'numeral'):
            wordsInSents[token.text] = 'determinant'
    
    dummy = {}
    removals = []
    for i in range(1,len(wordsInSents)):
        if list(wordsInSents.values())[i] != list(wordsInSents.values())[i-1]:
            dummy[list(wordsInSents.keys())[i-1]] = list(wordsInSents.values())[i-1]
        else:
            dummy[list(wordsInSents.keys())[i-1]+' '+list(wordsInSents.keys())[i]] = list(wordsInSents.values())[i]
            removals.append(i)
    
    if list(wordsInSents.values())[-1] == list(wordsInSents.values())[-2]:
        dummy[list(wordsInSents.keys())[-2]+' '+list(wordsInSents.keys())[-1]] = list(wordsInSents.values())[-1]
        dummy.pop(list(wordsInSents.keys())[-2], None)
    else:
        dummy[list(wordsInSents.keys())[-1]] = list(wordsInSents.values())[-1]
    
    for k in removals:
        dummy.pop(list(wordsInSents.keys())[k], None)
    
    wordsInSents = dummy
    
    #for i in range(1, len(wordsInSents)):
    #    if list(wordsInSents.keys())[i] == list(wordsInSents.keys())[i-1][-len(list(wordsInSents.keys())[i]):]:
    #        wordsInSents.pop(list(wordsInSents.keys())[i], None)
    
    return wordsInSents

In [40]:
list_of_wordsInSent_TA = []
for item in list_matched_sents:
    list_of_wordsInSent.append(decisionMaker(item))

In [25]:
with open('list_of_wordsInSent_TA.pkl', 'wb') as file:
     pickle.dump(list_of_wordsInSent_TA,file)

In [41]:
list_of_wordsInSent_TA

[]

In [ ]:
#with open('saved_list.pkl', 'wb') as f:
#    pickle.dump(list_of_dicts, f)
        
#with open('saved_list.pkl', 'rb') as f:
#    loaded_list_of_dicts = pickle.load(f)

In [ ]:
def check_bboxes(word, table_bbox):
    """
    Check whether word is inside a table bbox.
    """
    l = word['x0'], word['top'], word['x1'], word['bottom']
    r = table_bbox
    return l[0] > r[0] and l[1] > r[1] and l[2] < r[2] and l[3] < r[3]

tables = []
counter = 0
with pdfplumber.open("_inputs/S3 Tool Changer.pdf") as pdf:
    lines = [[] for x in range(len(pdf.pages))]
    for page in pdf.pages:        
        tables.append(page.find_tables())
        table_bboxes = [i.bbox for i in tables[counter]]
        tables[counter] = [{'table': i.extract(), 'doctop': i.bbox[1]} for i in tables[counter]]
        non_table_words = [word for word in page.extract_words() if not any(
            [check_bboxes(word, table_bbox) for table_bbox in table_bboxes])]

        for cluster in pdfplumber.utils.cluster_objects(non_table_words+tables[counter], 'doctop', tolerance=5):
            if 'text' in cluster[0]:
                lines[counter].append(' '.join([i['text'] for i in cluster]))
            elif 'table' in cluster[0]:
                lines[counter].append(cluster[0]['table'])
        counter+=1